In [8]:
import itertools
import numpy as np

from scipy.sparse import csr_matrix

from utilities import chol_params_to_lower_triangular_matrix
from utilities import cov_matrix_to_sdcorr_params
from utilities import number_of_triangular_elements_to_dimension
from utilities import dimension_to_number_of_triangular_elements
from utilities import cov_params_to_matrix

from utilities import commutation_matrix
from utilities import elimination_matrix
from utilities import duplication_matrix

from estimagic.optimization.utilities import robust_cholesky

from jax import jacfwd
from kernel_transformations_jax import covariance_to_internal as covariance_to_internal_jax 
from kernel_transformations_jax import sdcorr_to_internal as sdcorr_to_internal_jax
from kernel_transformations_jax import probability_to_internal as probability_to_internal_jax

from numpy.testing import assert_array_almost_equal

## Derivative of ``covariance_to_internal``

In [219]:
def derivative_covariance_to_internal(external_values):
    cov = cov_params_to_matrix(external_values)
    chol = robust_cholesky(cov)
    
    dim = len(chol)
    
    K = commutation_matrix(dim)
    
    d0_a = np.eye(dim ** 2) + K
    d0_b = np.kron(chol, np.eye(dim))
    d0 = d0_a @ d0_b
    
    L = elimination_matrix(dim)
    D = duplication_matrix(dim, L)
    
    d1 = L @ d0 @ D
    
    dfin = np.linalg.inv(d1)
    
    return dfin

## Example / Testing

In [6]:
def get_external_random(dim, seed=0):
    np.random.seed(seed)
    X = np.random.randn(dim, 1000)
    cov = np.cov(X)
    external = cov[np.tril_indices(dim)]
    return external

In [221]:
J = jacfwd(covariance_to_internal_jax)

In [223]:
for dim in range(10, 50):
    external = get_external_random(dim)
    jax_deriv = J(external)
    my_deriv = derivative_covariance_to_internal(external)
    assert_array_almost_equal(jax_deriv, my_deriv)

## Timeit

In [225]:
external = get_external_random(20)

In [226]:
%timeit J(external)

60.3 ms ± 2.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [227]:
%timeit derivative_covariance_to_internal(external)

20.6 ms ± 555 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Derivative of ``sdcorr_to_internal``

In [1]:
from derivatives import derivative_sdcorr_from_internal
from derivatives import derivative_sdcorr_to_internal

In [2]:
from kernel_transformations_jax import sdcorr_to_internal

In [4]:
from jax import jacfwd

In [5]:
J = jacfwd(sdcorr_to_internal)

In [9]:
external = get_external_random(2)

In [15]:
from utilities import sdcorr_params_to_matrix

In [16]:
cov = sdcorr_params_to_matrix(external)

In [19]:
bad = []
for dim in range(10, 20):
    try:
        external = get_external_random(dim)
        jax_deriv = J(external)
        deriv = derivative_sdcorr_to_internal(external)
        assert_array_almost_equal(deriv, jax_deriv)
    except Exception:
        bad.append(dim)
        
print(dim)

19


In [3]:
def transformation_matrix(dim):
    n = dimension_to_number_of_triangular_elements(dim)
    M = np.zeros((dim, dim)) + np.nan
    M[np.diag_indices(dim)] = np.arange(dim, dtype=int)
    M[np.tril_indices(dim, k=-1)] = np.arange(dim, n, dtype=int)
    
    m = M.ravel('F')
    num_na = np.count_nonzero(np.isnan(m))
    indices = m.argsort()[:-num_na]

    rows = [unit_vector_or_zeros(i, dim ** 2) for i in indices]
    
    transformer = np.row_stack(rows)
    return transformer

In [342]:
cov = np.cov(np.random.randn(10, 4))

In [346]:
sds, corr = cov_to_sds_and_corr(cov)

corr[np.diag_indices(len(cov))] = sds

vec = corr.ravel('F')

In [345]:
external = cov_matrix_to_sdcorr_params(cov)

In [349]:
(transformation_matrix(len(cov)) @ vec == external).all()

True

## Examples / Testing

## Timeit

## Derivative of ``probability_to_internal``

Let $f: \mathbb{R}^m \to \mathbb{R}^m, x \mapsto \frac{1}{x_m} x$. It is easily seen that

$$
J(f)(x) = \frac{1}{x_m} \sum_{k=1}^{m-1} e_k e_k^\top - 
\frac{1}{x_m^2} \left [0, \dots, 0, \left ( \begin{matrix} x_{1:m-1} \\ 0 \end{matrix} \right ) \right ]
$$
where $e_k$ denotes the $m$ dimensional unit vector.

In [229]:
def derivative_probability_to_internal(external_values):
    dim = len(external_values)
    
    deriv = np.eye(dim) / external_values[-1]
    deriv[:, -1] -= external_values / (external_values[-1] ** 2)
    deriv[-1, -1] = 0
    
    return deriv

## Examples / Testing

In [230]:
J = jacfwd(probability_to_internal_jax)

In [231]:
for dim in range(10, 50):
    external = get_external_random(dim)
    jax_deriv = J(external)
    my_deriv = derivative_probability_to_internal(external)
    assert_array_almost_equal(jax_deriv, my_deriv)

## Timeit

In [13]:
%timeit J(external)

4.6 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%timeit derivative_probability_to_internal(external)

17.9 µs ± 98.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
